In [ ]:
! pip install transformers
! pip install datasets

In [2]:
import torch
import numpy as np

from transformers import (AutoTokenizer, LongformerForSequenceClassification,
                          TrainingArguments, )
from datasets import load_dataset

In [12]:
model = LongformerForSequenceClassification.from_pretrained(
    'allenai/longformer-base-4096', num_labels = 11, problem_type="multi_label_classification")
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")
dataset = load_dataset("coastalcph/fairlex", 'ecthr')

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weigh

  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
def tokenize_function(examples):
    text = examples["text"]
    encoding = tokenizer(text, padding="max_length", truncation=True)
    labels_matrix = np.zeros((len(text), 11))
    for idx, label_list in enumerate(examples['labels']):
        if len(label_list) == 0:
            labels_matrix[idx, 10] = 1
        else:
            for label in label_list:
                labels_matrix[idx, label] = 1
    encoding["labels"] = labels_matrix.tolist()
    return encoding

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset['train'].column_names)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]